In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests

In [0]:
# 아래 링크에서 추출한 톹게이트의 영업소 코드 불러오기
# https://github.com/festivaleve/pj-1/blob/master/%ED%86%A8%EA%B2%8C%EC%9D%B4%ED%8A%B8_%EB%AA%A9%EB%A1%9D_%EC%B6%94%EC%B6%9C.ipynb
a =pd.read_csv('구분코드.csv',
               names=['도공민자구분코드','코드명','개방구분','개방타입코드','영업소코드','영업소명'])
b = a['영업소코드'].values

In [0]:
key ='보안키'

In [0]:
location=[]
for i in b:
  url = 'http://data.ex.co.kr/exopenapi/locationinfo/locationinfoUnit?serviceKey='+key+'&type=xml&unitCode='+'%d'%i+'&numOfRows=10&pageNo=1'
  res = requests.get(url)
  rss = BeautifulSoup(res.text, 'lxml')
  html = BeautifulSoup(res.text, 'lxml')
  for load in rss.find_all('list'):
    try:
      unitcode              = load.unitcode.text
      unitname             = load.unitname.text
      routeno       = load.routeno.text
      routename   = load.routename.text
      xvalue         = load.xvalue.text
      yvalue        = load.yvalue.text
      location.append([unitcode,unitname,routeno,routename,xvalue,yvalue])
    except AttributeError:  
      pass

In [24]:
c=pd.DataFrame(location)
c.to_csv('톨게이트 위치.csv',header=False, index= False)
d=pd.read_csv('톨게이트 위치.csv',names=['영업소코드','영업소명','노선코드','노선명','x좌표','y좌표'])
d.head()

,영업소코드,영업소명,노선코드,노선명,x좌표,y좌표
0,101,서울,1,경부선,127.102083,37.365071
1,102,동수원,50,영동선,127.047035,37.298106
2,103,수원신갈,1,경부선,127.102373,37.266859
3,104,지곡,35,중부선-대전통영선A,127.768313,35.590254
4,105,기흥,1,경부선,127.102439,37.222249


## 시각화

In [0]:
import folium as fl
from folium import plugins 

In [0]:
e=[]
for i in range(0,len(d['x좌표'])):
    e.append(d['y좌표'][i])
    e.append(d['x좌표'][i])   

In [0]:
e = np.array(e)
e.shape
e = e.reshape(350,2)
popups = [i for i in d['영업소명']]

In [27]:
m = fl.Map(
location=[36.2002, 127.054],
tiles='cartodbpositron',
zoom_start=7,
)
plugins.MarkerCluster(e,popups).add_to(m)
m